In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Let's give the dataset a small EDA , before we proceed to building a model and submitting the solution 

### Usually I tend to take the following steps

1) Finding the  n/p ratio and deciding how to move ahead , 
         n -> no of observations
         p -> no of predictors
         
2) Seeing the distribution of Data / Detecting Outliers 

3) Mean / Meadin / Mode of Target Variable

4) Handling Missing Values

5) Correlation

In [ ]:
df = pd.read_csv('/kaggle/input/santander-value-prediction-challenge/train.csv')
df.head()

# As we have a large number of columns , i.e 4993 .. Let's go forward and check the entire datashape , so that we get an understanding of what is our n/p ratio

In [ ]:
df.set_index("ID", inplace = True)

In [ ]:
df.head()

In [ ]:
print(df.shape)

# The rows = 4459  (N)
# The columns = 4993 (P)

The N/P < 1 .. so from my understanding we can't fit a very flexible(complex) model to the data , as it would go forward and overfit by great margins , as the provided data is very very small.

Or we need to find a way to drop a large number or columns (predictors) before throwing a fancy algorithm at it.

### Usually I tend to take the following steps

1) Finding the  n/p ratio and deciding how to move ahead , 
         n -> no of observations
         p -> no of predictors
         
      1.1) Removing rows with all 0's
      1.2) Removing duplicate columns
      1.3) Removing columns with constant value
      1.4) Removing correlated variables
         
         
2) Seeing the distribution of Data / Detecting Outliers 


3) Mean / Meadin / Mode of Target Variable

4) Handling Missing Values

5) Correlation

### Task 1.1 ) Removing columns with less than no values as non - zero

In [ ]:
df = df[(df.T != 0).any()]
df.shape

There are no such columns , with all zeroes

### Task 1.2 ) Removing duplicate columns

In [ ]:
df = df.loc[:,~df.columns.duplicated()]
df.shape

There are no such columns , with all duplicate values

### Task 1.3 ) Removing constant value columns , or its better to call them features with 0 variance

In [ ]:
train_features, test_features, train_labels, test_labels=train_test_split(df.drop(labels=['target'], axis=1),df['target'],test_size=0.2,
    random_state=41)

In [ ]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(train_features)

In [ ]:
len(train_features.columns[constant_filter.get_support()])

In [ ]:
train_features = constant_filter.transform(train_features)
test_features = constant_filter.transform(test_features)

train_features.shape, test_features.shape

In [ ]:
train_features

# THE TEST DATA WITH 49K VALUES ALSO HAS TO GO THROUGH THE SAME PROCESS , SO I WILL DEAL WITH IT HERE AND NOW

In [ ]:
test_df = pd.read_csv('/kaggle/input/santander-value-prediction-challenge/test.csv')

In [ ]:
test_df.shape
test_df = constant_filter.transform(test_df)

In [ ]:
test_df.shape()

We removed many columns , precisely (256) columns were removed . But still the no or predictors are more than the number of rows or the number of observations 

### Task 1.4 ) Removing variable which have correlation to other predictor variables

In [ ]:
X = df.drop(columns = "target")
y = df['target']
num_colums = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = list(X.select_dtypes(include=num_colums).columns)
X = X[numerical_columns]
X.shape

In [ ]:
correlated_features = set()
correlation_matrix = X.corr()

In [ ]:
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
X.drop(labels=correlated_features, axis=1, inplace=True)
X.shape

## TASK 2 - Studying Data Distribution & Outliers - 

Thanks to an amazing kernal ->https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-santander-value ,  navigating this section was really easy

Task 2.1) Detecting Outliers

Task 2.2) Studying Distribution Of the Data

Task 2.3) Fixing the target data

Task 2.4) Studying Mean Median Mode of new target


### Task 2.1) Detecting Outliers

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(data=df ,x=range(df['target'].shape[0]), y=np.sort(df["target"].values))

There are no visible outliers

### Task 2.2) Studying Distribution of Target Variable 

In [ ]:
ax = sns.distplot(df['target'])

In [ ]:
(df['target']).skew()

In [ ]:
ax = sns.distplot(np.log1p(df['target']))

In [ ]:
(np.log1p(df['target']).skew())

## The skew went down from 2 to -0.4

### More exploration of the target variable

In [ ]:
print(np.log1p(df['target']).value_counts())

In [ ]:
df['target'] = np.log1p(df['target'])

In [ ]:
y =  (df['target'])

# TASK 5 -> reducing variables using correlation

# There will be various feature selection processes , I would experiment with in the task-5 Section , I'll also train a default Gradient Boosting Regression Model for checking the quality of each method's progess . Creating good quality function is key here .

 * 1) Univariate Selection
 * 2) Feature Importance
 * 3) Correlation Heatmap
 * 4) PCA

# Univariate Selection


In [ ]:
#apply SelectKBest class to extract top 10 best features
#bestfeatures = SelectKBest(score_func=chi2, k=10)


In [ ]:
#fit = bestfeatures.fit(X,y)

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.75)
pca.fit(X)
df_pca_dropped = pca.transform(X)

In [ ]:
X.shape

In [ ]:
df_pca_dropped.shape

In [ ]:
df_pca_dropped = pd.DataFrame(df_pca_dropped)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


In [ ]:
y = y
X = df_pca_dropped
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
model.score(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print(r2_score(y_test,y_pred))

In [ ]:
X_train.head()

# Dropping columns using PCA gave us about r sqaure of 0.20 (GBM) AND 0.18(Random Forest) looks like PCA is dropping out on a lot of information:(

# There are some algorithms like lasso , decision trees which themselves choose b/w the large amount of variables and use the most suitable ones , let's go forward and train them directly on the initial full data